In [20]:
import sys
import time
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import socket
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder\
            .appName("John1")\
            .master("spark://login1.cluster:7077")\
            .config("spark.executor.instances", "1")\
            .config("spark.executor.cores", "11")\
            .config("spark.executor.memory", "60g")\
            .getOrCreate()
spark.conf.set('spark.sql.caseSensitive', True)
sc = spark.sparkContext
sc.setLogLevel("INFO")  # change this to WARN for less verbosity
print('======== START ========')
start = time.time()
print("Hello from ", socket.gethostname())
print("======== ELAPSED TIME: ", time.time() - start)

======== START ========
Hello from  login1.cluster
======== ELAPSED TIME:  0.0002815723419189453


/home/jkarasev/.local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [19]:
spark.stop()

In [23]:
# connecting the dir with data  
# part_009 3.85GB of data
raw_data = sc.textFile("file:///scratch/jkarasev/data-copy/works/updated_date=2024-01-10/part_009")

In [3]:
sc.version

'3.5.1'

In [24]:
# looking at string
raw_data.take(1)

['{"id": "https://openalex.org/W4252316661", "doi": "https://doi.org/10.1163/156973108x333803", "doi_registration_agency": "Crossref", "display_name": "Editorial", "title": "Editorial", "publication_year": 2008, "publication_date": "2008-01-01", "language": "en", "ids": {"openalex": "https://openalex.org/W4252316661", "doi": "https://doi.org/10.1163/156973108x333803"}, "primary_location": {"source": {"id": "https://openalex.org/S81350151", "issn_l": "1569-7312", "issn": ["1872-5163", "1569-7312"], "display_name": "Journal of Reformed Theology", "publisher": "Brill", "host_organization": "https://openalex.org/P4310320561", "host_organization_name": "Brill", "host_organization_lineage": ["https://openalex.org/P4310320561"], "host_organization_lineage_names": ["Brill"], "is_oa": false, "is_in_doaj": false, "host_institution_lineage": [], "host_institution_lineage_names": [], "publisher_lineage": ["https://openalex.org/P4310320561"], "publisher_lineage_names": ["Brill"], "publisher_id": "h

In [25]:
# converting to dic
import json
rdd = raw_data.map(json.loads)
# rdd.persist() # can't get it to save in memory

In [6]:
# show 2
rdd.take(2)

[{'id': 'https://openalex.org/W4252316661',
  'doi': 'https://doi.org/10.1163/156973108x333803',
  'doi_registration_agency': 'Crossref',
  'display_name': 'Editorial',
  'title': 'Editorial',
  'publication_year': 2008,
  'publication_date': '2008-01-01',
  'language': 'en',
  'ids': {'openalex': 'https://openalex.org/W4252316661',
   'doi': 'https://doi.org/10.1163/156973108x333803'},
  'primary_location': {'source': {'id': 'https://openalex.org/S81350151',
    'issn_l': '1569-7312',
    'issn': ['1872-5163', '1569-7312'],
    'display_name': 'Journal of Reformed Theology',
    'publisher': 'Brill',
    'host_organization': 'https://openalex.org/P4310320561',
    'host_organization_name': 'Brill',
    'host_organization_lineage': ['https://openalex.org/P4310320561'],
    'host_organization_lineage_names': ['Brill'],
    'is_oa': False,
    'is_in_doaj': False,
    'host_institution_lineage': [],
    'host_institution_lineage_names': [],
    'publisher_lineage': ['https://openalex.org

In [28]:
rdd.count()

468005

In [8]:
rdd.persist()

PythonRDD[5] at RDD at PythonRDD.scala:53

In [10]:
years = rdd.map(lambda obj: obj.get('publication_year'))

# Flatten the list of years and get unique values
unique_years = years.flatMap(lambda x: x).distinct().collect()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 58 in stage 5.0 failed 4 times, most recent failure: Lost task 58.3 in stage 5.0 (TID 450) (10.1.1.38 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/scratch/jkarasev/spark-3.5.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/scratch/jkarasev/spark-3.5.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/home/jkarasev/.local/lib/python3.7/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/jkarasev/.local/lib/python3.7/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/jkarasev/.local/lib/p